In [ ]:
import pandas as pd
#from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt
import tensorflow.keras.backend as K
import tensorflow as tf
import numpy as np
import cv2

from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense,Input,Lambda

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
from functools import partial
from albumentations import (
    Compose, RandomBrightness, JpegCompression, HueSaturationValue, RandomContrast, HorizontalFlip,Normalize,RandomBrightnessContrast,
    VerticalFlip,
    Rotate
)
import pandas as pd
#from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt
import tensorflow.keras.backend as K
import tensorflow as tf
import numpy as np
import cv2

from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense,Input,Lambda

#from albumentations import *
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [ ]:
pictureCsv = pd.read_csv("../input/mnist-zeroshot-learning/trainPictureArray.csv")
sigmoidTrain = pd.read_csv("../input/mnist-zeroshot-learning/pictureIndexData.csv")

In [ ]:
train = pd.read_csv("../input/trainvalsplit/trainSplit.csv")
validation = pd.read_csv("../input/trainvalsplit/validationSplit.csv")

In [ ]:
pictureCsvNoLabel = pictureCsv.iloc[:,:-1]
pictureCsvNoLabel = tf.convert_to_tensor(pictureCsvNoLabel.to_numpy())

In [ ]:
firstPictureIndex = train["firstPictureIndex"].values
secondPictureIndex = train["secondPictureIndex"].values

firstPictureIndexValidation = validation["firstPictureIndex"].values
secondPictureIndexValidation = validation["secondPictureIndex"].values

In [ ]:
image = pictureCsv.iloc[0][:-1].to_numpy()
imageV2 = np.stack([image,image,image])

In [ ]:
pictureCsv

In [ ]:
pixels = image.reshape((28, 28))

        # Plot
plt.title('Label is {label}'.format(label=2))
plt.imshow(pixels, cmap='gray')
plt.show()

In [ ]:
transforms = Compose([
            HorizontalFlip(),
            VerticalFlip(),
            #Normalize(),
            #RandomBrightnessContrast(),
           Normalize()
        ])

In [ ]:
def preprocess_image(index,process_type = "train",target_shape = (224,224)):
    """
    Load the specified file as a JPEG image, preprocess it and
    resize it to the target shape.
    """
    image = tf.reshape(pictureCsvNoLabel[index],(28,28))
    image = tf.stack([image, image, image],axis=2)
    #image = tf.image.decode_jpeg(image_string, channels=3)
    image = tf.reshape(image,(28,28,3))*255

    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize(image, target_shape)
    
    if process_type == "train":
        image = process_Augment_data(image,img_size = target_shape[0])
        
    return image

def aug_fn(image, img_size):
    data = {"image":image}
    aug_data = transforms(image=image)
    aug_img = aug_data["image"]
    aug_img = tf.image.resize(aug_img, size=[img_size, img_size])
    return aug_img

def process_Augment_data(image, img_size):
    aug_img = tf.numpy_function(func=aug_fn, inp=[image, img_size], Tout=tf.float64)
    return aug_img

def preprocess_siamese(positive, negative):
    """
    Given the filenames corresponding to the three images, load and
    preprocess them.
    """

    return (
        preprocess_image(positive),
        preprocess_image(negative),
    )

In [ ]:
imageFinalize = preprocess_image(0)

In [ ]:
image = pictureCsv.iloc[0][:-1].to_numpy().reshape((28,28))
imageV2 = np.stack([image,image,image],axis=2)
#imageV2 = imageV2.reshape((28,28,3))
#imageV2 = np.swapaxes(imageV2,0,2)
resized = cv2.resize(imageV2, (224,224), interpolation = cv2.INTER_AREA)
resized = resized*255

In [ ]:
import cv2

In [ ]:
for i in range(1):  
  plt.subplot(330 + 1 + i)
  plt.imshow(imageV2 , cmap=plt.get_cmap('gray'))
  plt.show()

Add Data Augmentation

In [ ]:
transforms = Compose([
            HorizontalFlip(),
            VerticalFlip(),
           Normalize()
        ])

val_transforms = Compose([
           Normalize()
        ])

In [ ]:
def preprocess_image(index,process_type = "train",target_shape = (224,224)):
    """
    Load the specified file as a JPEG image, preprocess it and
    resize it to the target shape.
    """
    image = tf.reshape(pictureCsvNoLabel[index],(28,28))
    image = tf.stack([image, image, image],axis=2)
    #image = tf.image.decode_jpeg(image_string, channels=3)
    image = tf.reshape(image,(28,28,3))*255

    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize(image, target_shape)
    
    if process_type == "train":
        image = process_Augment_data(image,img_size = target_shape[0])
    elif process_type == "validation":
        image = process_Augment_data_validation(image,img_size = target_shape[0])
    return image

def preprocess_image_test(index,process_type = "train",target_shape = (224,224)):
    """
    Load the specified file as a JPEG image, preprocess it and
    resize it to the target shape.
    """
    image = tf.reshape(testTensor[index],(28,28))
    image = tf.stack([image, image, image],axis=2)
    #image = tf.image.decode_jpeg(image_string, channels=3)
    image = tf.reshape(image,(28,28,3))*255

    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize(image, target_shape)
    
    if process_type == "train":
        image = process_Augment_data(image,img_size = target_shape[0])
    elif process_type == "validation":
        image = process_Augment_data_validation(image,img_size = target_shape[0])
    return image

def aug_fn(image, img_size):
    data = {"image":image}
    aug_data = transforms(image=image)
    aug_img = aug_data["image"]
    aug_img = tf.image.resize(aug_img, size=[img_size, img_size])
    return aug_img

def aug_fn_test(image, img_size):
    data = {"image":image}
    aug_data = val_transforms(image=image)
    aug_img = aug_data["image"]
    aug_img = tf.image.resize(aug_img, size=[img_size, img_size])
    return aug_img

def process_Augment_data(image, img_size):
    aug_img = tf.numpy_function(func=aug_fn, inp=[image, img_size], Tout=tf.float32)
    return aug_img

def process_Augment_data_validation(image, img_size):
    aug_img = tf.numpy_function(func=aug_fn_test, inp=[image, img_size], Tout=tf.float32)
    return aug_img

def preprocess_siamese(positive, negative,mode="train"):
    """
    Given the filenames corresponding to the three images, load and
    preprocess them.
    """

    return (
        preprocess_image(positive,mode),
        preprocess_image(negative,mode),
    )

In [ ]:
def preprocessEmbeddingTest(positive,mode="validation"):
    """
    Given the filenames corresponding to the three images, load and
    preprocess them.
    """

    return (
        preprocess_image_test(positive,mode)
    )

In [ ]:
def contrastive_loss(y, preds, margin=1):
	# explicitly cast the true class label data type to the predicted
	# class label data type (otherwise we run the risk of having two
	# separate data types, causing TensorFlow to error out)
	y = tf.cast(y, preds.dtype)
	# calculate the contrastive loss between the true labels and
	# the predicted labels
	squaredPreds = K.square(preds)
	squaredMargin = K.square(K.maximum(margin - preds, 0))
	loss = K.mean(y * squaredPreds + (1 - y) * squaredMargin)
	# return the computed contrastive loss to the calling function
	return loss

In [ ]:
sigmoidTrain["symmetrylabel"].values

In [ ]:
def create_dataset(listFirstPictureIndex,listSecondPictureIndex,batchsize,labels,useType="train"):
    FirstIndex = listFirstPictureIndex
    SecondIndex = listSecondPictureIndex
    dataset1 = tf.data.Dataset.from_tensor_slices(FirstIndex)
    dataset2 = tf.data.Dataset.from_tensor_slices(SecondIndex)
    labelDatasets = tf.data.Dataset.from_tensor_slices(labels)
    
    dataset = tf.data.Dataset.zip((dataset1, dataset2))
    dataset = dataset.map(lambda x,y:preprocess_siamese(x,y,useType))
    
    dataset = tf.data.Dataset.zip((dataset,labelDatasets))
    dataset = dataset.batch(batchsize, drop_remainder=False).prefetch(8)
    
    return dataset

In [ ]:
def create_datasetTest(pictureEmbeddingIndex,batchsize,useType="validation"):
    FirstIndex = pictureEmbeddingIndex
    dataset = tf.data.Dataset.from_tensor_slices(FirstIndex)
    
    dataset = dataset.map(lambda x:preprocessEmbeddingTest(x))
    
    dataset = dataset.batch(batchsize, drop_remainder=False).prefetch(8)
    
    return dataset

In [ ]:
train["symmetrylabel"].values

In [ ]:
train_ds = create_dataset(firstPictureIndex,secondPictureIndex,64,train["symmetrylabel"].values,"train")
val_ds = create_dataset(firstPictureIndexValidation,secondPictureIndexValidation,64,validation["symmetrylabel"].values,"validation")

In [ ]:
"""positive_images = firstPictureIndex 
negative_images = secondPictureIndex

image_count = len(positive_images )

positive_dataset = tf.data.Dataset.from_tensor_slices(positive_images)
negative_dataset = tf.data.Dataset.from_tensor_slices(negative_images)
label_dataset = tf.data.Dataset.from_tensor_slices(sigmoidTrain["symmetrylabel"].values)
# To generate the list of negative images, let's randomize the list of
# available images and concatenate them together.

#negative_images = triplettrain["Negative"].values
#np.random.RandomState(seed=32).shuffle(negative_images)

#negative_dataset = tf.data.Dataset.from_tensor_slices(negative_images)
#negative_dataset = negative_dataset.shuffle(buffer_size=4096)

dataset = tf.data.Dataset.zip((positive_dataset, negative_dataset))
#dataset = dataset.shuffle(buffer_size=1024)
dataset = dataset.map(preprocess_siamese)

dataset = tf.data.Dataset.zip((dataset,label_dataset))
dataset = dataset.shuffle(buffer_size=1024)

# Let's now split our dataset in train and validation.
train_dataset = dataset.take(round(image_count * 0.8))
val_dataset = dataset.skip(round(image_count * 0.8))

train_dataset = train_dataset.batch(32, drop_remainder=False)
train_dataset = train_dataset.prefetch(8)

val_dataset = val_dataset.batch(32, drop_remainder=False)
val_dataset = val_dataset.prefetch(8)"""

In [ ]:
for data in train_ds.take(3):
  break

In [ ]:
for data in val_ds.take(3):
  break

In [ ]:
def euclidean_distance(vectors):
	# unpack the vectors into separate lists
	(featsA, featsB) = vectors
	# compute the sum of squared distances between the vectors
	sumSquared = K.sum(K.square(featsA - featsB), axis=1,keepdims=True)
	# return the euclidean distance between the vectors
	return K.sqrt(K.maximum(sumSquared, K.epsilon()))

In [ ]:
input1 = tf.keras.Input(shape=(224,224,3))
input2 = tf.keras.Input(shape=(224,224,3))
effinetB0 = EfficientNetB0(include_top=False,weights="imagenet")(input1)
effinetB0_siamese = EfficientNetB0(include_top=False,weights="imagenet")(input2)
pooling1= tf.keras.layers.GlobalAveragePooling2D()(effinetB0)
pooling2 = tf.keras.layers.GlobalAveragePooling2D()(effinetB0_siamese)
distance = Lambda(euclidean_distance)([pooling1, pooling2])

firstEffinet = tf.keras.Model(
    inputs=input1, outputs=pooling1,name = "effinetB0"
)

twinEffinet = tf.keras.Model(
    inputs = input2,outputs= pooling2,name="effinetB0_siamese"
)

inputA = tf.keras.Input(shape=(224,224,3))
inputB = tf.keras.Input(shape=(224,224,3))

featureFirst = firstEffinet(inputA)
featureSecond = twinEffinet(inputB)

distance = Lambda(euclidean_distance)([featureFirst , featureSecond ])
siamese_network = Model(inputs=[inputA,inputB], outputs=distance)

In [ ]:
siamese_network.get_weights()[-1].mean()

In [ ]:
siamese_network.load_weights("../input/k/polapob/siamese-constructive-loss/siamesemodel05-validationloss0.03.h5")
siamese_network.get_weights()[-1].mean()

In [ ]:
firstEffinet.get_weights()[-1].mean()

In [ ]:
siamese_network.load_weights("../input/k/polapob/siamese-constructive-loss/siamesemodel07-validationloss0.02.h5")

In [ ]:
firstEffinet.get_weights()[-1].mean()

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
testData = pd.read_csv("../input/mnist-zeroshot-learning/testMnist.csv")
testPictureIndex = testData.index.values

In [ ]:
testTensor = tf.convert_to_tensor(testData.iloc[:,1:-1].to_numpy())
testTensor

In [ ]:
"""trainPrediction = []
for data in tqdm(create_datasetTest(firstPictureIndex,256)):
    trainPrediction.append(firstEffinet.predict(data))"""

In [ ]:
testPrediction = []
for data in tqdm(create_datasetTest(testPictureIndex,256)):
    testPrediction.append(firstEffinet.predict(data))

In [ ]:
testPredictionEmbedding = [eachPrediction for preds in testPrediction for eachPrediction in preds]

In [ ]:
myTestEmbedding = pd.DataFrame(testPredictionEmbedding,columns=[f"Embedding{i}" for i in range(1280)])
myTestEmbedding["label"] = testData["label"]
myTestEmbedding.to_csv("myTestEmbedding.csv",index=False)

Visualize data

In [ ]:
!pip install swat

In [ ]:
import swat
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from timeit import timeit

In [ ]:
from __future__ import print_function
import time
import numpy as np
import pandas as pd
#from sklearn.datasets import fetch_mldata
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns

In [ ]:
def VisualizePCA(csvFile,n_embedding = 256):
  speechRepresentation2 = pd.read_csv(csvFile)
  data_subset = speechRepresentation2.iloc[:,:n_embedding].to_numpy()
  pca = PCA(n_components=3)
  pca_result = pca.fit_transform(data_subset)
  df_subset2 = pd.DataFrame()
  df_subset2['pca-one'] = pca_result[:,0]
  df_subset2['pca-two'] = pca_result[:,1] 
  df_subset2['pca-three'] = pca_result[:,2]
  print('Explained variation per principal component: {}'.format(pca.explained_variance_ratio_))
  df_subset2["label"] = speechRepresentation2["label"]

  ax = plt.figure(figsize=(16,10)).gca(projection='3d')
  ax.scatter(
    xs=df_subset2["pca-one"], 
    ys=df_subset2["pca-two"], 
    zs=df_subset2["pca-three"], 
    c=df_subset2["label"], 
    cmap='tab10'
)
  ax.set_xlabel('pca-one')
  ax.set_ylabel('pca-two')
  ax.set_zlabel('pca-three')
  plt.show()

In [ ]:
from sklearn.datasets import load_digits
X, y = load_digits().data, load_digits().target
from cuml.manifold import TSNE
speechRepresentation2 = pd.read_csv("./myTestEmbedding.csv")
data_subset = speechRepresentation2.iloc[:,:1280].to_numpy()

y = speechRepresentation2["label"].values
tsne = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=300)
tsne_results = tsne.fit_transform(data_subset)

# To plot the embedding
import matplotlib.pyplot as plt
%matplotlib inline
plt.scatter(tsne_results[:,0], tsne_results[:,1], c = y, s = 0.5)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
cdict = {5: 'red', 6: 'blue', 7: 'green',8:"lightblue",9:"orange"}
fig, ax = plt.subplots()
group = y 
scatter_x = tsne_results[:,0]
scatter_y = tsne_results[:,1]

for g in np.unique(group):
    ix = np.where(group == g)
    ax.scatter(scatter_x[ix], scatter_y[ix], c = cdict[g], label = g, s = 0.5)
ax.legend()
plt.show()
#plt.scatter(tsne_results[:,0], tsne_results[:,1], c = y, s = 0.5)


In [ ]:
testDataV2 = pd.DataFrame({"embedding1":scatter_x,"embedding2":scatter_y,"label":y})

In [ ]:
testDataV2.to_csv("./myTestEmbedding2.csv")

In [ ]:
def VisualizeTSNE(csvFile,n_embedding = 256):
  speechRepresentation2 = pd.read_csv(csvFile)
  data_subset = speechRepresentation2.iloc[:,:n_embedding].to_numpy()
    
  tsne = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=300)
  tsne_results = tsne.fit_transform(data_subset)
  print("Waiting F***ing long time.")
  y = speechRepresentation2["label"].values

  plt.scatter(tsne_results[:,0], tsne_results[:,1], c = y, s = 0.5)
  plt.show()

In [ ]:
VisualizeTSNE("./myTestEmbedding.csv",n_embedding = 1280)

In [ ]:
VisualizePCA("./myTestEmbedding.csv",n_embedding = 1280)

In [ ]:
testData["label"]

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.utils.vis_utils import plot_model

plot_model(siamese_network, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
featureFirst = firstEffinet(inputA)

In [ ]:
model1 = Model()

In [ ]:
inputA = tf.keras.Input(shape=(224,224,3))
featureFirst = firstEffinet(inputA)

In [ ]:
def contrastive_loss(y, preds, margin=1):
	# explicitly cast the true class label data type to the predicted
	# class label data type (otherwise we run the risk of having two
	# separate data types, causing TensorFlow to error out)
	y = tf.cast(y, preds.dtype)
	# calculate the contrastive loss between the true labels and
	# the predicted labels
	squaredPreds = K.square(preds)
	squaredMargin = K.square(K.maximum(margin - preds, 0))
	loss = K.mean(y * squaredPreds + (1 - y) * squaredMargin)
	# return the computed contrastive loss to the calling function
	return loss

In [ ]:
siamese_network.summary()

In [ ]:
def preprocess_siamese_test(positive, negative,mode="validation"):
    """
    Given the filenames corresponding to the three images, load and
    preprocess them.
    """

    return (
        preprocess_image_test(positive,mode),
        preprocess_image_test(negative,mode),
    )

In [ ]:
def create_dataset_Similarity(listFirstPictureIndex,listSecondPictureIndex,batchsize,labels,useType="train"):
    FirstIndex = listFirstPictureIndex
    SecondIndex = listSecondPictureIndex
    dataset1 = tf.data.Dataset.from_tensor_slices(FirstIndex)
    dataset2 = tf.data.Dataset.from_tensor_slices(SecondIndex)
    labelDatasets = tf.data.Dataset.from_tensor_slices(labels)
    
    dataset = tf.data.Dataset.zip((dataset1, dataset2))
    dataset = dataset.map(lambda x,y:preprocess_siamese_test(x,y,useType))
    
    dataset = tf.data.Dataset.zip((dataset,labelDatasets))
    dataset = dataset.batch(batchsize, drop_remainder=False).prefetch(8)
    
    return dataset

In [ ]:
testSimilarity = pd.read_csv("../input/testpredictionmnistdata/TestPredictionData.csv")

In [ ]:
testSimilarity.drop_duplicates(inplace=True)
testSimilarity.reset_index(drop=True,inplace=True)

In [ ]:
testSimilarity

In [ ]:
testdsv2 = create_dataset_Similarity(testSimilarity["ImageA"],testSimilarity["ImageB"],64,testSimilarity["label"].values,"validation")

In [ ]:
prediction = []
for data,label in tqdm(testdsv2):
    prediction.append(siamese_network.predict(data))

In [ ]:
similarityPred = [eachvalue for listValue in prediction for eachvalue in listValue]

In [ ]:
testSimilarity["distance"]=[j for i in similarityPred for j in i]

In [ ]:
testSimilarity

In [ ]:
testSimilarity["predLabel"] = testSimilarity["distance"].apply(lambda x:0 if x > 1 else 1) 

In [ ]:
testSimilarity[testSimilarity["predLabel"]==testSimilarity["label"]].label.value_counts()

In [ ]:
accuracy = (testSimilarity["predLabel"] == testSimilarity["label"]).sum()/testSimilarity.shape[0] * 100
print(accuracy)

In [ ]:
def preprocess_image(index,process_type = "train",target_shape = (224,224)):
    """
    Load the specified file as a JPEG image, preprocess it and
    resize it to the target shape.
    """
    image = tf.reshape(pictureCsvNoLabel[index],(28,28))
    image = tf.stack([image, image, image],axis=2)
    #image = tf.image.decode_jpeg(image_string, channels=3)
    image = tf.reshape(image,(28,28,3))*255

    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize(image, target_shape)
    
    if process_type == "train":
        image = process_Augment_data(image,img_size = target_shape[0])
    elif process_type == "validation":
        image = process_Augment_data_validation(image,img_size = target_shape[0])
    return image

def preprocess_image_test(index,process_type = "train",target_shape = (224,224)):
    """
    Load the specified file as a JPEG image, preprocess it and
    resize it to the target shape.
    """
    image = tf.reshape(testTensor[index],(28,28))
    image = tf.stack([image, image, image],axis=2)
    #image = tf.image.decode_jpeg(image_string, channels=3)
    image = tf.reshape(image,(28,28,3))*255

    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize(image, target_shape)
    
    if process_type == "train":
        image = process_Augment_data(image,img_size = target_shape[0])
    elif process_type == "validation":
        image = process_Augment_data_validation(image,img_size = target_shape[0])
    return image

def aug_fn(image, img_size):
    data = {"image":image}
    aug_data = transforms(image=image)
    aug_img = aug_data["image"]
    aug_img = tf.image.resize(aug_img, size=[img_size, img_size])
    return aug_img

def aug_fn_test(image, img_size):
    data = {"image":image}
    aug_data = val_transforms(image=image)
    aug_img = aug_data["image"]
    aug_img = tf.image.resize(aug_img, size=[img_size, img_size])
    return aug_img

def process_Augment_data(image, img_size):
    aug_img = tf.numpy_function(func=aug_fn, inp=[image, img_size], Tout=tf.float32)
    return aug_img

def process_Augment_data_validation(image, img_size):
    aug_img = tf.numpy_function(func=aug_fn_test, inp=[image, img_size], Tout=tf.float32)
    return aug_img

def preprocess_siamese(positive, negative,mode="train"):
    """
    Given the filenames corresponding to the three images, load and
    preprocess them.
    """

    return (
        preprocess_image(positive,mode),
        preprocess_image(negative,mode),
    )

In [ ]:
testSimilarity = pd.read_csv("../input/mnist-zeroshot-learning/testMnist.csv")
testSimilarity 

In [ ]:
label = []
testSimilarity.index.values

In [ ]:
testSimilarity 

In [ ]:
train_data = testSimilarity 
allTrainIndex = testSimilarity.index.values

In [ ]:
from tqdm.notebook import tqdm
import random

In [ ]:
newDataSample = [20000,20000,20000,20000,20000]
oneData = train_data[train_data["label"] == 1]
checkAnchorPictureIndex = []
checkPositivePictureIndex = []
checkNegativePictureIndex = []
for number in tqdm(range(len(newDataSample))):
  numberOfDataAdded = newDataSample[number]
  addData = train_data[train_data["label"] == number + 5]
  negativeData = train_data[train_data["label"] != number + 5]
  for i in range(int(numberOfDataAdded)):
    while True:
      a = random.randint(0,len(allTrainIndex)-1)
      b = random.randint(0,len(allTrainIndex)-1)
      c = random.randint(0,len(allTrainIndex)-1)
      if a in addData.index and b in addData.index and a != b and c in negativeData.index:
        checkAnchorPictureIndex.append(a)
        checkPositivePictureIndex.append(b)
        checkNegativePictureIndex.append(c)
        break

In [ ]:
testTriplePicture = pd.DataFrame({"Anchor":checkAnchorPictureIndex,
                                   "Positive":checkPositivePictureIndex,
                                   "Negative":checkNegativePictureIndex}
                                  )

In [ ]:
testSimilarity.iloc[[5720,23662,28271],:]

In [ ]:
testTriplePicture["label"] = 1
anotherSample = testTriplePicture[["Positive","Negative"]]

In [ ]:
anotherSample["label"] = 0

In [ ]:
firstSample = testTriplePicture[["Anchor","Positive","label"]]
firstSample.columns = ["ImageA","ImageB","label"]

In [ ]:
firstSample

In [ ]:
anotherSample

In [ ]:
testData = pd.concat([anotherSample,firstSample],axis=0).reset_index(drop=True)

In [ ]:
testData.to_csv("TestPredictionData.csv",index=False)

In [ ]:
anotherSample.columns = ["ImageA","ImageB","label"]